In [ ]:
# Import packages
import sys

sys.path.append("../")

from db import models
from pathlib import Path
import os
from run import get_meta, get_closest
import cv2
import matplotlib.pyplot as plt
import exifread
import math
from math import radians, cos
from shapely import wkt
from PIL import Image
import yaml
from src.azure_blobs import download_image, get_blobs_by_folder_name
from tqdm import tqdm
from pyproj import Proj, transform
import numpy as np


IMAGE_FORMATS = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

In [ ]:
# Load inspection points from MSS
points = models.InspectionPoints.get_all()

In [ ]:
points = [
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.210578105000025 54.669064553000055 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.210214251000025 54.666455928000062 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.213230295000074 54.66648613600006 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.215971724000042 54.667454513000052 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.213579355000036 54.668781099000057 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.213854018000063 54.666440354000031 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.214523154000062 54.665463582000029 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.21330881800003 54.668866521000041 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.216072566000037 54.667924711000069 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.219072620000077 54.66725008700007 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.217168831000038 54.667331791000038 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.22837938300006 54.670533938000062 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.231419224000035 54.669863769000074 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.234452026000042 54.669177494000053 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.237247998000043 54.668073854000056 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.236598843000024 54.66506682000005 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.234903389000067 54.662459315000035 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.233214254000075 54.659845464000057 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.241896934000067 54.670487712000067 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.244625486000075 54.670502462000059 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.247492269000077 54.669292875000053 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.234787164000068 54.661607133000075 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.236450145000049 54.664214408000078 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.237948364000033 54.666933061000066 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.24619387000007 54.669967735000057 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.249066655000036 54.668776481000066 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.243215148000047 54.737331117000053 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245677990000047 54.735463511000034 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.248716216000048 54.734888441000066 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.251779305000071 54.734412747000079 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.253958437000051 54.732449770000073 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.254363205000061 54.729455332000043 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.253825599000038 54.726389298000072 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.253706634000025 54.723330898000029 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.255693686000029 54.720971520000035 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.25732615000004 54.718399003000059 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.251574876000063 54.734741950000057 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.254516501000069 54.727443407000067 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.254004644000076 54.724374590000082 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.255481723000059 54.721757362000062 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.257166246000054 54.719294017000038 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.258042413000055 54.716440876000036 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.241951253000025 54.730333973000029 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245031565000033 54.730689491000078 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.247937880000052 54.731650506000051 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.250786524000034 54.732896738000079 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.253739910000036 54.73387311700003 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.256692864000058 54.734856332000049 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245821158000069 54.727499519000048 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.244919282000069 54.724589889000072 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245881435000058 54.72168094400007 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.248644781000053 54.720388375000027 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.251753267000026 54.720307133000063 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.254840810000076 54.720637662000058 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245426566000049 54.727387014000044 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.244517212000062 54.72444646100007 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245581193000078 54.721576420000076 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.24832174900007 54.720229449000044 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.251426018000075 54.720109925000031 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.254512291000026 54.720413220000069 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.255863457000032 54.734332552000069 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.252920857000049 54.733324949000064 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.249972648000039 54.732331437000028 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.24715801800005 54.731006191000063 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.244150253000043 54.73026906900003 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.241052807000074 54.729996390000053 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.236599187000024 54.732283592000044 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.238736969000058 54.73009122600007 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.240621197000053 54.727635345000067 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.242261556000074 54.725021703000039 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.243790582000031 54.72231957300005 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.245448544000055 54.719714513000042 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.247650520000036 54.717874815000073 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.25009357600004 54.716731500000037 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.252878441000064 54.716132868000045 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.255982090000032 54.716352338000036 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.267832216000045 54.737601174000076 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.270802337000077 54.738511026000026 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.273793160000025 54.73932921100004 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.276304885000059 54.739820519000034 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.276805926000065 54.74281854000003 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.267878493000069 54.737383750000049 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.270851666000056 54.738300650000042 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.273862197000028 54.739032812000062 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.276711919000036 54.738769429000058 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.277907522000078 54.735916123000038 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.278842578000024 54.732969666000031 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.279633673000035 54.729977466000037 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.277054212000053 54.74194633500008 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.277152984000054 54.739001741000038 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.278332079000052 54.73612115800006 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.279266778000078 54.733170159000053 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.280076822000069 54.730164799000079 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.283336854000027 54.702503213000057 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.28406693900007 54.705482950000032 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.285304968000048 54.708330576000037 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.286157048000064 54.711319788000083 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.286635207000074 54.714333448000048 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.284056247000024 54.714483660000042 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.281163633000062 54.713353234000067 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.278079949000073 54.712944485000037 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.275009369000031 54.71267272700004 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.271940750000056 54.712220983000066 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.26964076400003 54.710273764000078 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.268024165000043 54.707614909000029 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.266682272000025 54.704897613000071 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.264654805000077 54.703165895000041 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.265981582000052 54.703293631000065 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.26886132900006 54.704417227000079 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.27195729500005 54.704451978000066 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.275060146000044 54.70420299500006 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.278159585000026 54.703975467000078 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.281258638000054 54.70369733800004 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.280487219000065 54.704038523000065 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.277383591000046 54.704262443000061 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.274284225000031 54.704544086000055 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.271186435000061 54.704835597000056 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.26818675800007 54.704703461000065 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.274973396000064 54.701359839000077 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.273833006000075 54.698566468000081 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.275385020000044 54.701344744000039 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.274262160000035 54.698551863000034 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.254405399000063 54.71611542800008 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.283611238000049 54.702422870000078 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.284282193000024 54.705234160000032 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.285561863000055 54.708069440000031 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.286471961000075 54.710917373000029 0)",
    },
    {
        "object": 1,
        "object_name": "-",
        "manager": None,
        "Shape": "POINT (25.287042927000073 54.713970661000076 0)",
    },
    {
        "object": 2,
        "object_name": "Gerosios Vilties st.",
        "manager": None,
        "Shape": "POINT (25.241928597000026 54.670811885000035 0)",
    },
    {
        "object": 2,
        "object_name": "Gerosios Vilties st.",
        "manager": None,
        "Shape": "POINT (25.241603259000044 54.670373852000068 0)",
    },
    {
        "object": 2,
        "object_name": "Linkmenų st.",
        "manager": None,
        "Shape": "POINT (25.269852659000037 54.704861120000032 0)",
    },
    {
        "object": 2,
        "object_name": "Linkmenų st.",
        "manager": None,
        "Shape": "POINT (25.271964429000036 54.704534282000054 0)",
    },
    {
        "object": 2,
        "object_name": "Kalvarijų turgus",
        "manager": None,
        "Shape": "POINT (25.282505277000041 54.699343082000041 0)",
    },
    {
        "object": 2,
        "object_name": "Kalvarijų turgus",
        "manager": None,
        "Shape": "POINT (25.283037861000025 54.700211547000038 0)",
    },
    {
        "object": 2,
        "object_name": "Šnipiškės",
        "manager": None,
        "Shape": "POINT (25.269035839000026 54.699905862000037 0)",
    },
    {
        "object": 2,
        "object_name": "Žalgirio st.",
        "manager": None,
        "Shape": "POINT (25.283487831000059 54.703119752000077 0)",
    },
    {
        "object": 2,
        "object_name": "Žalgirio st.",
        "manager": None,
        "Shape": "POINT (25.282806276000031 54.703935595000075 0)",
    },
    {
        "object": 2,
        "object_name": "Giedraičių st.",
        "manager": None,
        "Shape": "POINT (25.284880752000049 54.707335543000056 0)",
    },
    {
        "object": 2,
        "object_name": "Tauragnų st.",
        "manager": None,
        "Shape": "POINT (25.286206907000064 54.711510574000044 0)",
    },
    {
        "object": 2,
        "object_name": "Pramogų arena",
        "manager": None,
        "Shape": "POINT (25.286615008000069 54.714476756000067 0)",
    },
    {
        "object": 2,
        "object_name": "Licėjus",
        "manager": None,
        "Shape": "POINT (25.279016040000045 54.713024767000036 0)",
    },
    {
        "object": 2,
        "object_name": "Žalgirio st.",
        "manager": None,
        "Shape": "POINT (25.282198969000035 54.70373365100005 0)",
    },
    {
        "object": 2,
        "object_name": "Tauragnų st.",
        "manager": None,
        "Shape": "POINT (25.28676123200006 54.712419262000026 0)",
    },
    {
        "object": 2,
        "object_name": "Broniaus Laurinavičiaus skveras",
        "manager": None,
        "Shape": "POINT (25.284079768000026 54.704112648000034 0)",
    },
    {
        "object": 2,
        "object_name": "Žalgirio st.",
        "manager": None,
        "Shape": "POINT (25.284298018000072 54.704379262000032 0)",
    },
    {
        "object": 2,
        "object_name": "Giedraičių st.",
        "manager": None,
        "Shape": "POINT (25.285531576000039 54.70804214900005 0)",
    },
    {
        "object": 2,
        "object_name": "Šiltnamių žiedas",
        "manager": None,
        "Shape": "POINT (25.221478096000055 54.66707787200005 0)",
    },
    {
        "object": 2,
        "object_name": "Miškiniai",
        "manager": None,
        "Shape": "POINT (25.226382064000063 54.670912342000065 0)",
    },
    {
        "object": 2,
        "object_name": "Šiltnamių st.",
        "manager": None,
        "Shape": "POINT (25.214871698000024 54.67089573800007 0)",
    },
    {
        "object": 2,
        "object_name": "Pakalnučių st.",
        "manager": None,
        "Shape": "POINT (25.213121303000037 54.669102706000047 0)",
    },
    {
        "object": 2,
        "object_name": "Bijūnų st.",
        "manager": None,
        "Shape": "POINT (25.215942354000049 54.667402587000026 0)",
    },
    {
        "object": 2,
        "object_name": "Bijūnų st.",
        "manager": None,
        "Shape": "POINT (25.215981678000048 54.667112650000036 0)",
    },
    {
        "object": 2,
        "object_name": "Gelvonėlio st.",
        "manager": None,
        "Shape": "POINT (25.256715259000032 54.714372130000072 0)",
    },
    {
        "object": 2,
        "object_name": "Gelvonėlio st.",
        "manager": None,
        "Shape": "POINT (25.257445102000077 54.715086757000051 0)",
    },
    {
        "object": 2,
        "object_name": "Paberžės",
        "manager": None,
        "Shape": "POINT (25.254895820000058 54.716159208000079 0)",
    },
    {
        "object": 2,
        "object_name": "Gelvonų st.",
        "manager": None,
        "Shape": "POINT (25.257600282000055 54.717637293000053 0)",
    },
    {
        "object": 2,
        "object_name": "Gelvonų st.",
        "manager": None,
        "Shape": "POINT (25.257671326000036 54.71843369000004 0)",
    },
    {
        "object": 2,
        "object_name": "Paberžės",
        "manager": None,
        "Shape": "POINT (25.253604873000029 54.716202451000072 0)",
    },
    {
        "object": 2,
        "object_name": "Ateities st.",
        "manager": None,
        "Shape": "POINT (25.266819843000064 54.73706734700005 0)",
    },
    {
        "object": 2,
        "object_name": "Kalvarijos",
        "manager": None,
        "Shape": "POINT (25.276840361000041 54.738512676000028 0)",
    },
    {
        "object": 2,
        "object_name": "Seminarija",
        "manager": None,
        "Shape": "POINT (25.277953010000033 54.736990483000056 0)",
    },
    {
        "object": 2,
        "object_name": "Seminarija",
        "manager": None,
        "Shape": "POINT (25.277960241000073 54.735788414000069 0)",
    },
    {
        "object": 2,
        "object_name": "Pušynas",
        "manager": None,
        "Shape": "POINT (25.278961308000078 54.734306511000057 0)",
    },
    {
        "object": 2,
        "object_name": "Pušynas",
        "manager": None,
        "Shape": "POINT (25.278761179000071 54.733347263000041 0)",
    },
    {
        "object": 2,
        "object_name": "Ateities st.",
        "manager": None,
        "Shape": "POINT (25.265963507000038 54.736003192000055 0)",
    },
    {
        "object": 2,
        "object_name": "Ateities st.",
        "manager": None,
        "Shape": "POINT (25.266556769000033 54.736299967000036 0)",
    },
    {
        "object": 2,
        "object_name": "Ateities st.",
        "manager": None,
        "Shape": "POINT (25.265179831000069 54.736871006000058 0)",
    },
    {
        "object": 2,
        "object_name": "Ateities st.",
        "manager": None,
        "Shape": "POINT (25.265867171000025 54.73746959500005 0)",
    },
    {
        "object": 2,
        "object_name": "Kalvarijos",
        "manager": None,
        "Shape": "POINT (25.275525554000069 54.739346535000038 0)",
    },
    {
        "object": 2,
        "object_name": "Kalvarijos",
        "manager": None,
        "Shape": "POINT (25.276809366000066 54.739876515000049 0)",
    },
    {
        "object": 2,
        "object_name": "Jeruzalė",
        "manager": None,
        "Shape": "POINT (25.276892276000069 54.743008857000063 0)",
    },
    {
        "object": 2,
        "object_name": "Jeruzalė",
        "manager": None,
        "Shape": "POINT (25.277875377000043 54.744122680000032 0)",
    },
    {
        "object": 2,
        "object_name": "Eigulių st.",
        "manager": None,
        "Shape": "POINT (25.235477268000068 54.668977692000055 0)",
    },
    {
        "object": 2,
        "object_name": "Vilkpėdės st.",
        "manager": None,
        "Shape": "POINT (25.236834816000055 54.664766173000032 0)",
    },
    {
        "object": 2,
        "object_name": "Vilkpėdės st.",
        "manager": None,
        "Shape": "POINT (25.235399416000064 54.663201874000038 0)",
    },
    {
        "object": 2,
        "object_name": "Riovonys",
        "manager": None,
        "Shape": "POINT (25.233580215000075 54.659611662000032 0)",
    },
    {
        "object": 2,
        "object_name": "Riovonys",
        "manager": None,
        "Shape": "POINT (25.232269912000049 54.658917344000031 0)",
    },
    {
        "object": 2,
        "object_name": "Vilkpėdės žiedas",
        "manager": None,
        "Shape": "POINT (25.24569850000006 54.665667969000026 0)",
    },
    {
        "object": 2,
        "object_name": "Ratilių st.",
        "manager": None,
        "Shape": "POINT (25.214254936000032 54.665120839000053 0)",
    },
    {
        "object": 2,
        "object_name": "Jurgio Baltrušaičio st.",
        "manager": None,
        "Shape": "POINT (25.238923786000043 54.729338479000035 0)",
    },
    {
        "object": 2,
        "object_name": "Jurgio Baltrušaičio st.",
        "manager": None,
        "Shape": "POINT (25.239613071000065 54.729030412000043 0)",
    },
    {
        "object": 2,
        "object_name": "Stasio Lozoraičio st.",
        "manager": None,
        "Shape": "POINT (25.242566702000033 54.724372994000078 0)",
    },
    {
        "object": 2,
        "object_name": "Fabijoniškės",
        "manager": None,
        "Shape": "POINT (25.237386204000074 54.731508043000076 0)",
    },
    {
        "object": 2,
        "object_name": "Takas",
        "manager": None,
        "Shape": "POINT (25.236842740000043 54.732941698000047 0)",
    },
    {
        "object": 2,
        "object_name": "Takas",
        "manager": None,
        "Shape": "POINT (25.236112390000073 54.733987407000029 0)",
    },
    {
        "object": 2,
        "object_name": "Fabijoniškių žiedas",
        "manager": None,
        "Shape": "POINT (25.241369654000039 54.739129563000063 0)",
    },
    {
        "object": 2,
        "object_name": "Sporto gimnazija",
        "manager": None,
        "Shape": "POINT (25.242327225000054 54.731599065000069 0)",
    },
    {
        "object": 2,
        "object_name": "Kaimelio žiedas",
        "manager": None,
        "Shape": "POINT (25.244468694000034 54.731561974000044 0)",
    },
    {
        "object": 2,
        "object_name": "Sporto gimnazija",
        "manager": None,
        "Shape": "POINT (25.244699232000073 54.732966194000028 0)",
    },
    {
        "object": 2,
        "object_name": "Pijaus Žadeikos st.",
        "manager": None,
        "Shape": "POINT (25.246189703000027 54.734854623000047 0)",
    },
    {
        "object": 2,
        "object_name": "Pijaus Žadeikos st.",
        "manager": None,
        "Shape": "POINT (25.247456391000071 54.73505719800005 0)",
    },
    {
        "object": 2,
        "object_name": "Antano Jonyno st.",
        "manager": None,
        "Shape": "POINT (25.24736030300005 54.731369373000064 0)",
    },
    {
        "object": 2,
        "object_name": "Antano Jonyno st.",
        "manager": None,
        "Shape": "POINT (25.248901085000057 54.731550320000053 0)",
    },
    {
        "object": 2,
        "object_name": "Liudo Giros st.",
        "manager": None,
        "Shape": "POINT (25.252098984000042 54.733325703000048 0)",
    },
    {
        "object": 2,
        "object_name": "Liudo Giros st.",
        "manager": None,
        "Shape": "POINT (25.253097769000078 54.734062985000037 0)",
    },
    {
        "object": 2,
        "object_name": "Liudo Giros st.",
        "manager": None,
        "Shape": "POINT (25.25368607300004 54.733089522000057 0)",
    },
    {
        "object": 2,
        "object_name": "Liudo Giros st.",
        "manager": None,
        "Shape": "POINT (25.254213590000063 54.733753123000042 0)",
    },
    {
        "object": 2,
        "object_name": "Pamiškė",
        "manager": None,
        "Shape": "POINT (25.254931060000047 54.73039747100006 0)",
    },
    {
        "object": 2,
        "object_name": "Pamiškė",
        "manager": None,
        "Shape": "POINT (25.254479012000047 54.729541063000056 0)",
    },
    {
        "object": 2,
        "object_name": "Simono Stanevičiaus st.",
        "manager": None,
        "Shape": "POINT (25.254207816000076 54.726195463000067 0)",
    },
    {
        "object": 2,
        "object_name": "Simono Stanevičiaus st.",
        "manager": None,
        "Shape": "POINT (25.253695779000054 54.725204304000044 0)",
    },
    {
        "object": 2,
        "object_name": "Jovaro st.",
        "manager": None,
        "Shape": "POINT (25.253982462000067 54.723800200000028 0)",
    },
    {
        "object": 2,
        "object_name": "Jovaro st.",
        "manager": None,
        "Shape": "POINT (25.254043296000077 54.722869121000031 0)",
    },
    {
        "object": 2,
        "object_name": "Pijaus Žadeikos st.",
        "manager": None,
        "Shape": "POINT (25.246294590000048 54.735509843000045 0)",
    },
    {
        "object": 2,
        "object_name": "Igno Šeiniaus st.",
        "manager": None,
        "Shape": "POINT (25.24429526800003 54.721444733000055 0)",
    },
    {
        "object": 2,
        "object_name": "Kaimelio st.",
        "manager": None,
        "Shape": "POINT (25.244422364000059 54.730584233000059 0)",
    },
    {
        "object": 2,
        "object_name": "Kaimelio žiedas",
        "manager": None,
        "Shape": "POINT (25.244940647000078 54.731497398000045 0)",
    },
    {
        "object": 2,
        "object_name": "Bijūnų st.",
        "manager": None,
        "Shape": "POINT (25.216733118000036 54.66749799400003 0)",
    },
    {
        "object": 2,
        "object_name": "Vilkpėdės žiedas",
        "manager": None,
        "Shape": "POINT (25.244948742000076 54.665723117000027 0)",
    },
    {
        "object": 2,
        "object_name": "Rugių st.",
        "manager": None,
        "Shape": "POINT (25.269332568000038 54.738056814000061 0)",
    },
    {
        "object": 2,
        "object_name": "Rugių st.",
        "manager": None,
        "Shape": "POINT (25.270375995000052 54.738133571000049 0)",
    },
    {
        "object": 2,
        "object_name": "Kaimelio st.",
        "manager": None,
        "Shape": "POINT (25.246305289000077 54.73069771400003 0)",
    },
    {
        "object": 2,
        "object_name": "Giraitės st.",
        "manager": None,
        "Shape": "POINT (25.237788802000068 54.668914173000076 0)",
    },
    {
        "object": 2,
        "object_name": "Giraitės st.",
        "manager": None,
        "Shape": "POINT (25.238249341000028 54.668700563000073 0)",
    },
    {
        "object": 2,
        "object_name": "Šiltnamių žiedas",
        "manager": None,
        "Shape": "POINT (25.221078516000034 54.666921736000063 0)",
    },
    {
        "object": 2,
        "object_name": "Krokuvos st.",
        "manager": None,
        "Shape": "POINT (25.273238094000078 54.698495825000066 0)",
    },
    {
        "object": 2,
        "object_name": "Krokuvos st.",
        "manager": None,
        "Shape": "POINT (25.274516044000052 54.698854271000073 0)",
    },
    {
        "object": 2,
        "object_name": "Kernavės st.",
        "manager": None,
        "Shape": "POINT (25.275813865000032 54.702156264000052 0)",
    },
    {
        "object": 2,
        "object_name": "Kernavės st.",
        "manager": None,
        "Shape": "POINT (25.274934853000047 54.701189148000026 0)",
    },
    {
        "object": 2,
        "object_name": "Oršos st.",
        "manager": None,
        "Shape": "POINT (25.276833443000044 54.704106254000067 0)",
    },
    {
        "object": 2,
        "object_name": "Šnipiškės",
        "manager": None,
        "Shape": "POINT (25.268860068000038 54.699463846000071 0)",
    },
    {
        "object": 3,
        "object_name": "Vilniaus Japoniškas sodas",
        "manager": None,
        "Shape": "POINT (25.268179897000039 54.701794244000041 0)",
    },
    {
        "object": 3,
        "object_name": "Vilniaus Japoniškas sodas",
        "manager": None,
        "Shape": "POINT (25.267015889000049 54.70339500800003 0)",
    },
    {
        "object": 3,
        "object_name": "Vilniaus Japoniškas sodas",
        "manager": None,
        "Shape": "POINT (25.266291726000077 54.701921411000058 0)",
    },
    {
        "object": 3,
        "object_name": "Neries senvagės parkas",
        "manager": None,
        "Shape": "POINT (25.271189408000055 54.708829068000057 0)",
    },
    {
        "object": 3,
        "object_name": "Neries senvagės parkas",
        "manager": None,
        "Shape": "POINT (25.270814144000042 54.70724896300004 0)",
    },
    {
        "object": 3,
        "object_name": "Neries senvagės parkas",
        "manager": None,
        "Shape": "POINT (25.270544396000048 54.710233035000044 0)",
    },
    {
        "object": 3,
        "object_name": "Neries senvagės parkas",
        "manager": None,
        "Shape": "POINT (25.270569820000048 54.710187931000064 0)",
    },
    {
        "object": 3,
        "object_name": "Neries senvagės parkas",
        "manager": None,
        "Shape": "POINT (25.271568871000056 54.710638677000077 0)",
    },
    {
        "object": 3,
        "object_name": "Neries senvagės parkas",
        "manager": None,
        "Shape": "POINT (25.272218577000046 54.71124416300006 0)",
    },
]

In [ ]:
# Define current path
current_path = Path().absolute()

In [ ]:
# Reading the configuration.yml file
with open(os.path.join(current_path,'..', "configuration.yml"), "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
images_input_dir = os.path.join(current_path, "input")
os.makedirs(images_input_dir, exist_ok=True)

In [ ]:
images_output_dir = os.path.join(current_path, "output")
os.makedirs(images_output_dir, exist_ok=True)

In [ ]:
storage_folder_name = (
    "RnD/Gatvių valymas/2023-2024/20231130 Ateieties-Jaruzales-saligatviai/"
)

In [ ]:
storage_images = get_blobs_by_folder_name(
    config=config, name_starts_with=storage_folder_name
)
# Downloading images from Azure storage to local dir
for img in tqdm(storage_images, desc="Downloading images from Azure Storage:"):
    download_image(blob_name=img, config=config, local_file_dir=images_input_dir)

In [ ]:
images = []
for root, dirs, files in os.walk(images_input_dir):
    for file in files:
        # Infering whether the file ends with .jpg, .JPG, .jpeg, .png, .PNG
        if file.endswith(IMAGE_FORMATS):
            # Adding the full path to the file
            file = os.path.join(root, file)
            # Appending to the list of images to infer
            images.append(file)

In [ ]:
# Extracting the function to extract the focal length
def extract_focal_length(file_path):
    # Open the image file for reading in binary mode
    with open(file_path, "rb") as f:
        # Read the EXIF data
        tags = exifread.process_file(f)

        # Check if GPSInfo tag is present
        if "EXIF FocalLength" in tags:
            # Extract latitude, longitude, and altitude
            focal_length = tags["EXIF FocalLength"].values[0]

            return float(focal_length)
        else:
            print("Focal length not found in the metadata.")
            return None

In [ ]:
def extract_gps_coordinates(file_path):
    # Open the image file for reading in binary mode
    with open(file_path, "rb") as f:
        # Read the EXIF data
        tags = exifread.process_file(f)

        # Check if GPSInfo tag is present
        if (
            "GPS GPSLatitude" in tags
            and "GPS GPSLongitude" in tags
            and "GPS GPSAltitude" in tags
        ):
            # Extract latitude, longitude, and altitude
            latitude = tags["GPS GPSLatitude"].values
            longitude = tags["GPS GPSLongitude"].values
            altitude = tags["GPS GPSAltitude"].values

            # Convert coordinates to decimal format
            latitude_decimal = latitude[0] + latitude[1] / 60 + latitude[2] / 3600
            longitude_decimal = longitude[0] + longitude[1] / 60 + longitude[2] / 3600

            return float(latitude_decimal), float(longitude_decimal), float(altitude[0])
        else:
            print("GPS information not found in the metadata.")
            return None

In [ ]:
# Defining the function that calculates the diff in pixels to meters
# GSD - Ground Sampling Distance
def calculate_gsd(
    height_from_ground,
    image_width,
    image_height,
    sensor_width,
    sensor_height,
    focal_length,
):
    """
    Function that calculates the GSD (Ground Sampling Distance) from pixels to meters

    Args:
        height_from_ground (float): Height from ground in meters
        image_width (int): Image width in pixels
        image_height (int): Image height in pixels
        sensor_width (float): Sensor width in mm
        sensor_height (float): Sensor height in mm
        focal_length (float): Focal length in mm

    Returns:
        gsd_h (float): Horizontal GSD in meters
        gsd_v (float): Vertical GSD in meters
    """
    # Calculating the horizontal and vertical GSD
    gsd_h = (height_from_ground * sensor_width) / (focal_length * image_width)
    gsd_v = (height_from_ground * sensor_height) / (focal_length * image_height)
    # Returning the average GSD
    return gsd_h, gsd_v

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences in coordinates
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = (
        math.sin(delta_lat / 2) ** 2
        + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2) ** 2
    )

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


def calculate_distances(point_A, point_B):
    x_distance = haversine_distance(
        (point_A[0] + point_B[0]) / 2,
        point_A[1],
        (point_A[0] + point_B[0]) / 2,
        point_B[1],
    )

    y_distance = haversine_distance(
        point_A[0],
        (point_A[1] + point_B[1]) / 2,
        point_B[0],
        (point_A[1] + point_B[1]) / 2,
    )

    return x_distance, y_distance

In [ ]:
def flat_earth_coordinates_difference(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calculate horizontal (x) and vertical (y) differences
    horizontal_difference = dlon * cos(lat1)
    vertical_difference = dlat

    # Assuming Earth's radius is 6371 km
    earth_radius = 6371.0

    # Convert differences to meters
    horizontal_difference_meters = horizontal_difference * earth_radius * 1000
    vertical_difference_meters = vertical_difference * earth_radius * 1000

    return horizontal_difference_meters, vertical_difference_meters

In [ ]:
def lonlat_to_utm(lon, lat):
    utm_projection = Proj(proj="utm", zone=33, ellps="WGS84", preserve_units=False)
    return transform(Proj(init="epsg:4326"), utm_projection, lon, lat)


def calculate_pixel_difference(lon1, lat1, lon2, lat2, gsd_x, gsd_y):
    x1, y1 = lonlat_to_utm(lon1, lat1)
    x2, y2 = lonlat_to_utm(lon2, lat2)

    distance_x = np.abs(x2 - x1)
    distance_y = np.abs(y2 - y1)

    pixel_diff_x = distance_x / gsd_x
    pixel_diff_y = distance_y / gsd_y

    return pixel_diff_x, pixel_diff_y

In [ ]:
# Define sensor width and sensor height
sensor_width = 10.16
sensor_height = 7.619999999999999

In [ ]:
for _img in images:
    # Get image center coordinates
    x, y = get_meta(_img)
    # Get closest inspection point
    closest_point = get_closest((x, y), points)
    # Open image with OpenCV
    img = Image.open(_img)
    img = np.array(img)
    img = img[:, :, ::-1].copy()
    # Get image height and width
    h, w = img.shape[0], img.shape[1]
    # Get rectangle with 25 pixel offset
    center_box = [w / 2 - 25, h / 2 - 25, w / 2 + 25, h / 2 + 25]
    center_box = [int(x) for x in center_box]
    #  Draw rectange on image center
    img = cv2.rectangle(
        img,
        (center_box[0], center_box[1]),
        (center_box[2], center_box[3]),
        (255, 0, 0),
        2,
    )
    # Get focal lenght from image metadata
    focal_length = extract_focal_length(_img)
    # Get image center coordinate and absoulute height
    y, x, z = extract_gps_coordinates(_img)
    # Load inspection point geometry
    point = wkt.loads(closest_point["Shape"])
    # Get xmp metadata from image
    xmp = Image.open(_img).getxmp()
    # Get relative height
    rel_alt = xmp["xmpmeta"]["RDF"]["Description"]["RelativeAltitude"]
    rel_alt = float(rel_alt)
    # Calculate gsd values
    gsd_h, gsd_v = calculate_gsd(
        rel_alt, w, h, sensor_width, sensor_height, focal_length
    )
    horizontal_distance, vertical_distance = flat_earth_coordinates_difference(
        y, x, point.y, point.x
    )

    # Convert meters to pixels
    horizontal_pixels = horizontal_distance / gsd_v
    vertical_pixels = vertical_distance / gsd_h

    # Get rectangle with 25 pixel offset
    addjusted_box = [
        (w / 2 - 25) + vertical_pixels,
        (h / 2 - 25) + horizontal_pixels,
        (w / 2 + 25) + vertical_pixels,
        (h / 2 + 25) + horizontal_pixels,
    ]
    addjusted_box = [int(x) for x in addjusted_box]

    #  Draw rectange on image center
    img = cv2.rectangle(
        img,
        (addjusted_box[0], addjusted_box[1]),
        (addjusted_box[2], addjusted_box[3]),
        (0, 0, 255),
        2,
    )

    cv2.imwrite(os.path.join(images_output_dir, os.path.basename(_img)), img)